In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import sys
from pathlib import Path
import collections

import pandas as pd
import pdfplumber


def extract_tables_from_pdf(pdf_path: Path):
    """
    Extrae todas las tablas de un PDF y las agrupa por encabezado.
    Devuelve un dict:
        { header_tuple: [df1, df2, ...] }
    donde header_tuple es una tupla con los nombres de las columnas.
    """
    grouped = collections.defaultdict(list)

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_tables = page.extract_tables() or []
            for t_idx, table in enumerate(page_tables, start=1):
                if not table:
                    continue

                df = pd.DataFrame(table)

                # Suponemos que la primera fila es el encabezado
                if df.shape[0] < 2:
                    continue  # casi seguro no es tabla útil

                df.columns = df.iloc[0].astype(str).str.strip()
                df = df[1:].reset_index(drop=True)

                # Normalizamos nombres de columnas
                df.columns = [str(c).strip() for c in df.columns]

                header_key = tuple(df.columns)
                grouped[header_key].append(df)

    return grouped


def pdf_to_excel(pdf_path: Path, output_dir: Path = None):
    """
    Convierte un PDF en un Excel con una hoja por grupo de tablas
    que tengan el mismo encabezado.
    """
    if output_dir is None:
        output_dir = pdf_path.parent

    output_dir.mkdir(parents=True, exist_ok=True)

    grouped_tables = extract_tables_from_pdf(pdf_path)

    if not grouped_tables:
        print(f"[ADVERTENCIA] No se encontraron tablas en: {pdf_path.name}")
        return None

    output_path = output_dir / f"{pdf_path.stem}.xlsx"

    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for i, (header, dfs) in enumerate(grouped_tables.items(), start=1):
            merged_df = pd.concat(dfs, ignore_index=True)
            sheet_name = f"Tabla_{i}"
            merged_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"[OK] {pdf_path.name} -> {output_path.name}")
    return output_path


def main():
    """
    Uso:
      - En terminal: 
            python pdf_a_excel_tablas.py archivo1.pdf archivo2.pdf
      - En Jupyter: simplemente ejecuta la celda, y buscará todos los .pdf
        de la carpeta actual, ignorando argumentos raros del kernel.
    """
    # Nos quedamos SOLO con argumentos que terminen en .pdf
    arg_pdfs = [a for a in sys.argv[1:] if a.lower().endswith(".pdf")]

    if arg_pdfs:
        pdf_paths = [Path(p) for p in arg_pdfs]
    else:
        # Si no hay PDFs en los argumentos, tomamos todos los .pdf del directorio
        pdf_paths = sorted(Path(".").glob("*.pdf"))

    if not pdf_paths:
        print("No encontré archivos PDF en la carpeta actual ni en los argumentos.")
        return

    print("Procesando PDFs:")
    for pdf_path in pdf_paths:
        if not pdf_path.is_file():
            print(f"[ERROR] No existe el archivo: {pdf_path}")
            continue
        pdf_to_excel(pdf_path)


if __name__ == "__main__":
    main()


Procesando PDFs:
[OK] 1er. evento de votación sobre participación de Cacif en Conadie.pdf -> 1er. evento de votación sobre participación de Cacif en Conadie.xlsx
[OK] 2o. evento de votación por participación del Cacif en Conadie.pdf -> 2o. evento de votación por participación del Cacif en Conadie.xlsx
